In [4]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import torch
import numpy as np
from assemble import Matrices3DSurface
from preconditioner import Preconditioner
from solver import ConjugateGradient
from visualize import Visualization3DSurface
from boundary import apply_dirichlet_boundary_conditions
import time
from ionic import ModifiedMS2v
from mesh.triangulation import Triangulation
from mesh.materialproperties import MaterialProperties
from mesh.stimulus import Stimulus
from tools import load_stimulus_region, dfmass, sigmaTens
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64
print(device)


dt = 0.01
vg = 0.1
diffusl = (1000 * 1000) * 0.175
diffust = (1000 * 1000) * 0.04375
tin = 0.15
tout = 1.5
topen = 105.0
tclose = 185.0

use_renumbering = True
T = 2400
nt = T // dt

cfgstim1 = {'tstart': 0.0,
            'nstim': 3,
            'period': 800,
            'duration': np.max([2.0, dt]),
            'intensity': 1.0,
            'name': 'S1'}


cuda


In [22]:
diffusl, diffust

(175000.0, 43750.0)

In [5]:
material = MaterialProperties()
material.add_element_property('sigma_l', 'uniform', diffusl)
material.add_element_property('sigma_t', 'uniform', diffust)
material.add_nodal_property('tau_in', 'uniform', tin)
material.add_nodal_property('tau_out', 'uniform', tout)
material.add_nodal_property('tau_open', 'uniform', topen)
material.add_nodal_property('tau_close', 'uniform', tclose)
material.add_nodal_property('u_gate', 'uniform', vg)
material.add_nodal_property('u_crit', 'uniform', vg)
material.add_ud_function('mass', dfmass)
material.add_ud_function('stiffness', sigmaTens)


domain = Triangulation()
domain.readMesh("/home/bzhou6/Projects/FinitePDE/data/Case_1")
# domain.exportCarpFormat("atrium")

# assign nodal properties
ionic_model = ModifiedMS2v()
nodal_properties = material.nodal_property_names()
point_region_ids = domain.point_region_ids()
npt = point_region_ids.shape[0]

for npr in nodal_properties:
    npr_type = material.nodal_property_type(npr)
    attribute_value = ionic_model.get_attribute(npr)

    if attribute_value is not None:
        if npr_type == "uniform":
            values = material.NodalProperty(npr, -1, -1)
        else:
            values = torch.full(size=(npt, 1), fill_value=attribute_value)
            for point_id, region_id in enumerate(point_region_ids):
                values[point_id] = material.NodalProperty(npr, point_id, region_id)
        ionic_model.set_attribute(npr, values)

adding the element property sigma_l of type uniform
adding the element property sigma_t of type uniform
adding the nodal property tau_in of type uniform
adding the nodal property tau_out of type uniform
adding the nodal property tau_open of type uniform
adding the nodal property tau_close of type uniform
adding the nodal property u_gate of type uniform
adding the nodal property u_crit of type uniform
reading Nodes
read 394613 nodes in 0.46 s.
reading Elements
read 780617 elements in 2.11 s.
reading Fibers
read 780617 Fibers in 0.71 s.
total: 3.28 s.
Associating a region ID to points
done in 4.08 s


'\ntau_in 0.15\ntau_out 1.5\ntau_open 105.0\ntau_close 185.0\nu_gate 0.1\nu_crit 0.1\n'

In [17]:
fib = domain.Fibres()
fib.shape

(780617, 3)

In [33]:
domain.Pts().shape

(394613, 3)

In [37]:
domain.Elems()['Trias']

array([[     0,      1,      2,      3],
       [     1,      3,      2,      3],
       [     4,      5,      6,      4],
       ...,
       [215664, 394603, 216358,      5],
       [394603, 216357, 216358,      5],
       [293939, 293941, 293944,      4]], dtype=int32)

In [20]:
domain.element_contravariant_basis('Trias', 0)

{'v1': array([ 0.00163637, -0.00680932, -0.00038617]),
 'v2': array([-0.00116806,  0.00268557,  0.00304068]),
 'v3': array([-0.95973457, -0.22078717, -0.17367375]),
 'meas': array([24398.51163918])}

In [36]:
domain.Pts()

array([[-69444.046875  , -44598.31640625, -31379.47851562],
       [-69434.046875  , -44750.6171875 , -31241.12304688],
       [-69497.59375   , -44630.26953125, -31042.953125  ],
       ...,
       [-59683.4296875 , -51202.91796875, -11705.01757812],
       [-68755.71875   , -38948.9765625 , -34955.2734375 ],
       [-61361.57421875, -49669.08203125, -37493.65625   ]])

In [39]:
vertices = torch.from_numpy(domain.Pts())
triangles = torch.from_numpy(domain.Elems()['Trias'][:, :-1])

In [61]:
import pyvista as pv
from pathlib import Path
class Visualization3D:
    def __init__(self, vertices, triangles):
        self.vertices = vertices
        self.triangles = triangles
        self.n_triangles = triangles.shape[0]

    def save_frame(self, color_values, frame_path):
        # Cells array indicating triangles (3 vertices per triangle)
        cells = np.hstack((np.full((self.n_triangles, 1), 3, dtype=int), self.triangles)).flatten().astype(int)
        # Cell type for triangles
        cell_type = np.full(self.n_triangles, 5, dtype=int)
        # Create the unstructured grid
        grid = pv.UnstructuredGrid(cells, cell_type, self.vertices.numpy())
        grid.point_data["colors"] = color_values

        # Save the grid to the specified file path
        file_path = Path(frame_path)
        file_path.parent.mkdir(parents=True, exist_ok=True)
        grid.save(frame_path)

In [62]:
Visualization3D(vertices, triangles).save_frame(np.random.rand(len(vertices)), "1.vtk")